In [1]:
import yfinance as yf
import datetime
import numpy as np
import pandas as pd

def get_stock_data(ticker, current_date, manual_interest_rate=None):
    # 計算に必要なデータの取得範囲を設定
    end_date = current_date
    start_date = end_date - datetime.timedelta(days=365)
    
    try:
        # 株価データを取得
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        
        # 1年前の価格と現在の価格を取得
        price_start = stock_data['Adj Close'].iloc[0]
        price_end = stock_data['Adj Close'].iloc[-1]
        
        # 利回りを手動で設定または計算
        if manual_interest_rate is not None:
            interest_rate = manual_interest_rate
        else:
            # 銘柄の利回りを取得
            interest_rate_data = get_interest_rate_data(start_date, end_date)
            if interest_rate_data is not None and not interest_rate_data.empty:
                interest_rate = interest_rate_data['GS10'].mean() / 100
            else:
                interest_rate = 0.02  # 利回りが取得できない場合、デフォルト値として0.02を使用
        
        # 現在の幾何平均を計算
        geometric_mean = np.sqrt(price_start * price_end)
        
        # 1年後の想定利益を計算
        expected_profit = price_end - geometric_mean
        
        #現在から先ほどの想定利益を加算して想定
        
        #多めに見積もるパターン１
        #expected_profit = price_end + expected_profit
        
        # 配当を計算 配当金の減量分をあらかじめ入れている
        dividend = price_end * (1 - interest_rate)
        
        #多めに見積もるパターン１
        #dividend = price_end * interest_rate
        
        # 1年後の想定全体利益を計算
        expected_total_profit = expected_profit + dividend
        
        # 想定利率を計算
        expected_interest_rate = expected_total_profit / price_end
        
        return {
                  "銘柄": ticker,
                  "始値（1年前）": price_start,
                  "終値（現在）": price_end,
                  "利回り": interest_rate,
                  "幾何平均": geometric_mean,
                  "1年後の想定利益": expected_profit,
                  "配当": dividend,
                  "1年後の想定総利益": expected_total_profit,
                  "想定利率": expected_interest_rate  
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None
    
def get_interest_rate_data(start_date, end_date):
    # 利回りデータを取得
    # ここではダミーデータを使用しますが、実際にはFREDなどのデータソースから取得してください
    dummy_data = {
        "Date": pd.date_range(start=start_date, end=end_date),
        "GS10": [2.0] * len(pd.date_range(start=start_date, end=end_date))
    }
    interest_rate_data = pd.DataFrame(dummy_data)
    return interest_rate_data

# テスト用の現在の日付
current_date = datetime.datetime(2024, 8, 13)

# 銘柄コードを指定してデータを取得
result = get_stock_data("6701.T", current_date)

if result is not None:
    print(result)


[*********************100%***********************]  1 of 1 completed
{'銘柄': '6701.T', '始値（1年前）': 6922.66162109375, '終値（現在）': 11865.501953125, '利回り': 0.02, '幾何平均': 9063.15921662592, '1年後の想定利益': 2802.3427364990803, '配当': 11628.1919140625, '1年後の想定総利益': 14430.53465056158, '想定利率': 1.2161756584398884}


In [25]:
def get_expected_interest_rates(tickers, manual_interest_rates=None):
    expected_interest_rates = []
    
    for i, ticker in enumerate(tickers):
        manual_interest_rate = None
        if manual_interest_rates is not None and i < len(manual_interest_rates):
            manual_interest_rate = manual_interest_rates[i]
        
        stock_data = get_stock_data(ticker, datetime.datetime.now(), manual_interest_rate)
        
        if stock_data is not None:
            expected_interest_rate = stock_data["想定利率"]
            expected_interest_rates.append(f"{ticker}の想定利率: {expected_interest_rate:.4f}")
    
    return expected_interest_rates

#テスト用のリスト1とリスト2
tickers = ["8604.T", "6702.T", "2768.T","4307.T"]
manual_interest_rates = [0.023, 0.0116, 0.039, 0.0125]  # 手動で設定する利回りのリスト

#関数を呼び出して想定利率を取得
result = get_expected_interest_rates(tickers, manual_interest_rates)
for item in result:
    print(item)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
8604.Tの想定利率: 1.0854
6702.Tの想定利率: 1.2188
2768.Tの想定利率: 1.0009
4307.Tの想定利率: 1.1459


In [5]:
# テスト用のリスト1とリスト2
tickers = ["1720.T"]
manual_interest_rates = [0.0477]  # 手動で設定する利回りのリスト

# 関数を呼び出して想定利率を取得
result = get_expected_interest_rates(tickers, manual_interest_rates)
for item in result:
    print(item)

[*********************100%***********************]  1 of 1 completed
1720.Tの想定利率: 0.9984


In [5]:
def extract_interest_rates(result):
    interest_rates = []
    
    for item in result:
        # 文字列を ":" で分割し、数値部分を取得
        parts = item.split(":")
        if len(parts) == 2:
            interest_rate = float(parts[1].strip())
            interest_rates.append(interest_rate)
    
    return interest_rates

# テスト用のリスト
result = [
"8604.Tの想定利率: 1.1990",
"6702.Tの想定利率: 1.1025",
"2768.Tの想定利率: 1.0637",
"7267.Tの想定利率: 1.0378",
"7272.Tの想定利率: 1.0276",
"4668.Tの想定利率: 1.0376",
]

# 関数を呼び出して数値データのみを取得
interest_rates = extract_interest_rates(result)
print(interest_rates)


[1.199, 1.1025, 1.0637, 1.0378, 1.0276, 1.0376]


In [6]:
def calculate_ratios(data):
    total = sum(data)
    ratios = [x / total for x in data]
    return ratios

def multiply_ratios(ratios, X):
    multiplied_ratios = [ratio * X for ratio in ratios]
    return multiplied_ratios

ratios = calculate_ratios(interest_rates)
print(ratios)

X=240 
result = multiply_ratios(ratios, X)
print(result)

[0.18536841779784174, 0.1704492749141956, 0.16445069725735134, 0.1604464920688909, 0.15886954639621534, 0.1604155715655051]
[44.48842027148202, 40.907825979406944, 39.46816734176432, 38.507158096533814, 38.12869113509168, 38.49973717572122]


In [2]:
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import re

csv_data = pd.read_csv("Nikkei225.csv")

nikkei225_ticker = []
for number in csv_data["Unnamed: 0"]:
    nikkei225_ticker.append(f"{number}.T")

# 結果を表示
print(nikkei225_ticker)

['1332.T', '1605.T', '1721.T', '1801.T', '1802.T', '1803.T', '1808.T', '1812.T', '1925.T', '1928.T', '1963.T', '2002.T', '2269.T', '2282.T', '2413.T', '2432.T', '2501.T', '2502.T', '2503.T', '2768.T', '2801.T', '2802.T', '2871.T', '2914.T', '3086.T', '3092.T', '3099.T', '3289.T', '3382.T', '3401.T', '3402.T', '3405.T', '3407.T', '3436.T', '3659.T', '3861.T', '3863.T', '4004.T', '4005.T', '4021.T', '4042.T', '4043.T', '4061.T', '4063.T', '4183.T', '4188.T', '4208.T', '4324.T', '4452.T', '4631.T', '4901.T', '4911.T', '6988.T', '4151.T', '4385.T', '4502.T', '4503.T', '4506.T', '4507.T', '4519.T', '4523.T', '4543.T', '4568.T', '4578.T', '4661.T', '4689.T', '4704.T', '4751.T', '4755.T', '4902.T', '5019.T', '5020.T', '5101.T', '5108.T', '5201.T', '5214.T', '5233.T', '5301.T', '5332.T', '5333.T', '5401.T', '5406.T', '5411.T', '5631.T', '5706.T', '5711.T', '5713.T', '5714.T', '5801.T', '5802.T', '5803.T', '5831.T', '6098.T', '6103.T', '6113.T', '6146.T', '6178.T', '6273.T', '6301.T', '6302.T',

In [3]:
import yfinance as yf
import numpy as np
import datetime

def get_stock_data(ticker, current_date):
    # 計算に必要なデータの取得範囲を設定
    end_date = current_date
    start_date = end_date - datetime.timedelta(days=365)
    
    try:
        # 株価データを取得
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        
        if len(stock_data) == 0:
            raise ValueError("データが見つかりませんでした")
        
        # 1年前の価格と現在の価格を取得
        price_start = stock_data['Adj Close'].iloc[0]
        price_end = stock_data['Adj Close'].iloc[-1]
        
        # 現在の幾何平均を計算
        geometric_mean = np.sqrt(price_start * price_end)
        
        # 1年後の想定利益を計算
        expected_profit = price_end - geometric_mean
        
        ecpected_price = price_end + expected_profit
        
        #計算方法②
        
        #expected_profit = geometric_mean - price_start
        
        #ecpected_price = price_end + expected_profit

        # 想定利率を計算
        expected_interest_rate = 1 + (expected_profit / price_end)
        
        return {
                  "銘柄": ticker,
                  "始値（1年前）": price_start,
                  "終値（現在）": price_end,
                  #"幾何平均成長率": geometric_mean,
                  "1年後の想定価格": ecpected_price,
                  "想定倍率": expected_interest_rate  
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# 現在の日付を取得
current_date = datetime.datetime.now()

# 関数のテスト
ticker = '1332.T'  # テスト用に日経225銘柄を使用
result = get_stock_data(ticker, current_date)

# 結果を表示
print(result)


[*********************100%***********************]  1 of 1 completed
{'銘柄': '1332.T', '始値（1年前）': 673.8793334960938, '終値（現在）': 936.5, '1年後の想定価格': 1078.5895293872998, '想定倍率': 1.1517240036169778}


In [4]:
# 銘柄コードを指定してデータを取得
result = get_stock_data("2768.T", current_date)
print(result)

[*********************100%***********************]  1 of 1 completed
{'銘柄': '2768.T', '始値（1年前）': 2954.750244140625, '終値（現在）': 3165.0, '1年後の想定価格': 3271.9312429729316, '想定倍率': 1.033785542803454}


In [5]:
csv_data["Unnamed: 0"] = nikkei225_ticker
csv_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,1332.T,ニッスイ,水産
1,1605.T,ＩＮＰＥＸ,鉱業
2,1721.T,コムシスＨＤ,建設
3,1801.T,大成建設,建設
4,1802.T,大林組,建設
...,...,...,...
220,9735.T,セコム,サービス
221,9766.T,コナミＧ,サービス
222,9843.T,ニトリＨＤ,小売業
223,9983.T,ファーストリテイリング,小売業


In [6]:
# 新しい列を作成（初期値はNaN）
csv_data["始値（1年前）"] = np.nan
csv_data["終値（現在）"] = np.nan
csv_data["1年後の想定価格"] = np.nan
csv_data["想定倍率"] = np.nan
csv_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,始値（1年前）,終値（現在）,1年後の想定価格,想定倍率
0,1332.T,ニッスイ,水産,NaN,NaN,NaN,NaN
1,1605.T,ＩＮＰＥＸ,鉱業,NaN,NaN,NaN,NaN
2,1721.T,コムシスＨＤ,建設,NaN,NaN,NaN,NaN
3,1801.T,大成建設,建設,NaN,NaN,NaN,NaN
4,1802.T,大林組,建設,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
220,9735.T,セコム,サービス,NaN,NaN,NaN,NaN
221,9766.T,コナミＧ,サービス,NaN,NaN,NaN,NaN
222,9843.T,ニトリＨＤ,小売業,NaN,NaN,NaN,NaN
223,9983.T,ファーストリテイリング,小売業,NaN,NaN,NaN,NaN


In [7]:
# 各ティッカーに対してget_stock_dataを実行し、データフレームに代入
for idx, ticker in enumerate(nikkei225_ticker):
    stock_data = get_stock_data(ticker, current_date)
    if stock_data:
        csv_data.at[idx, "始値（1年前）"] = stock_data["始値（1年前）"]
        csv_data.at[idx, "終値（現在）"] = stock_data["終値（現在）"]
        csv_data.at[idx, "1年後の想定価格"] = stock_data["1年後の想定価格"]
        csv_data.at[idx, "想定倍率"] = stock_data["想定倍率"]

# 結果を表示
print(csv_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [14]:
csv_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,始値（1年前）,終値（現在）,1年後の想定価格,想定倍率
0,1332.T,ニッスイ,水産,668.039001,924.400024,1062.965926,1.149898
1,1605.T,ＩＮＰＥＸ,鉱業,1925.359619,2083.500000,2164.130367,1.038699
2,1721.T,コムシスＨＤ,建設,2858.982422,3116.000000,3247.274011,1.042129
3,1801.T,大成建設,建設,4761.185059,6325.000000,7162.332418,1.132385
4,1802.T,大林組,建設,1184.289551,1802.500000,2143.944931,1.189429
...,...,...,...,...,...,...,...
220,9735.T,セコム,サービス,4852.524414,5340.000000,5589.569726,1.046736
221,9766.T,コナミＧ,サービス,7486.773438,14090.000000,17909.233829,1.271060
222,9843.T,ニトリＨＤ,小売業,15447.975586,20715.000000,23541.321618,1.136438
223,9983.T,ファーストリテイリング,小売業,31081.755859,48490.000000,58157.914255,1.199380


In [8]:
# CSVデータを出力するためのコード
output_csv_path = "Nikkei225_stock_data_2024_11.csv"

# データフレームをCSVファイルに保存
csv_data.to_csv(output_csv_path, index=False)

# ファイルパスを表示
print(f"CSVファイルが出力されました: {output_csv_path}")

CSVファイルが出力されました: Nikkei225_stock_data_2024_11.csv


In [ ]:
import yfinance as yf
import numpy as np
import datetime

def get_stock_data(ticker, current_date):
    # 計算に必要なデータの取得範囲を設定
    end_date = current_date
    start_date = end_date - datetime.timedelta(days=90)
    
    try:
        # 株価データを取得
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        
        if len(stock_data) == 0:
            raise ValueError("データが見つかりませんでした")
        
        # 1年前の価格と現在の価格を取得
        price_start = stock_data['Adj Close'].iloc[0]
        price_end = stock_data['Adj Close'].iloc[-1]
        
        # 現在の幾何平均を計算
        geometric_mean = np.sqrt(price_start * price_end)
        
        # 1年後の想定利益を計算
        #expected_profit = price_end - geometric_mean
        
        #ecpected_price = price_end + expected_profit
        
        #計算方法②
        
        expected_profit = geometric_mean - price_start
        
        ecpected_price = price_end + expected_profit

        # 想定利率を計算
        expected_interest_rate = 1 + (expected_profit / price_end)
        
        return {
                  "銘柄": ticker,
                  "始値（1年前）": price_start,
                  "終値（現在）": price_end,
                  #"幾何平均成長率": geometric_mean,
                  "3か月後の想定価格": ecpected_price,
                  "想定倍率": expected_interest_rate  
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# 現在の日付を取得
current_date = datetime.datetime.now()

csv_data = pd.read_csv("Nikkei225.csv")

nikkei225_ticker = []
for number in csv_data["Unnamed: 0"]:
    nikkei225_ticker.append(f"{number}.T")
    
csv_data["Unnamed: 0"] = nikkei225_ticker

# 新しい列を作成（初期値はNaN）
csv_data["始値（1年前）"] = np.nan
csv_data["終値（現在）"] = np.nan
csv_data["3か月後の想定価格"] = np.nan
csv_data["想定倍率"] = np.nan

for idx, ticker in enumerate(nikkei225_ticker):
    stock_data = get_stock_data(ticker, current_date)
    if stock_data:
        csv_data.at[idx, "始値（1年前）"] = stock_data["始値（1年前）"]
        csv_data.at[idx, "終値（現在）"] = stock_data["終値（現在）"]
        csv_data.at[idx, "3か月後の想定価格"] = stock_data["3か月後の想定価格"]
        csv_data.at[idx, "想定倍率"] = stock_data["想定倍率"]
        
# CSVデータを出力するためのコード
output_csv_path = "Nikkei225_stock_data_2024_10_3m.csv"

# データフレームをCSVファイルに保存
csv_data.to_csv(output_csv_path, index=False)

# ファイルパスを表示
print(f"CSVファイルが出力されました: {output_csv_path}")